## Test agency historic budget data

In [3]:
import sys
!{sys.executable} -m pip install PyAthena

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.4/75.4 kB 739.3 kB/s eta 0:00:00a 0:00:01


In [2]:
import awswrangler as wr
import pandas as pd
import datetime as dt
from datetime import datetime

## TIME PERIOD OF DATASET

In [157]:
#TIMEFRAME
start_date = '2021-01-01'
end_date = '2024-02-11'
###SHOULD BE DATED YESTERDAY, TO PREDICT TODAY AND SEVEN DAYS LATER###
###SE PARA EN EL 4 DE FEBRERO CON DATO CIERTO 3 DE FEBRERO Y AGREGA 8 DIAS###

### DFs generation

In [158]:
# DB Setting
bucket_name = 's3://viamericas-datalake-dev-us-east-1-283731589572-athena/'
origin_name = 'AwsDataCatalog'
database_name= 'viamericas'
table_name = 'daily_check_gp'

In [7]:
from pyathena import connect
import pandas as pd
conn = connect(s3_staging_dir='s3://aws-athena-query-results-283731589572-us-east-1',
               region_name='us-east-1')

df = pd.read_sql(" SELECT * FROM athenaquery."viamericas"."history_balance" WHERE id_main_branch = 'F568'order by day DESC;", conn)
df

SyntaxError: invalid syntax. Perhaps you forgot a comma? (3979677168.py, line 6)

In [159]:
df = wr.athena.read_sql_table(
    table=table_name,
    database=database_name,
)

In [160]:
df.day.max() # CHECK

'2024-02-03'

In [161]:
# Convert the 'date' column to datetime format
df['day'] = pd.to_datetime(df['day'])
# Grouping by 'payer' and 'country' concatenated for this level of granularity
df['payer_country'] = df['payer'] + '_' + df['country']
# Margin (when tx !=0)
df['margin'] = df.apply(lambda row: row['gp'] / row['tx'] if row['tx'] != 0 else 0, axis=1)
df['margin'] = df['margin'].apply(lambda x: float(x)).round(4)

In [162]:
# DATA RANGE
df = df[(df['date'] >= start_date) & (df['date'] <= end_date)]